## Neural Networks Processing Uncertain Information
### Theoretical background
**Continuous perceptron with interval excitation state**

This approach is a generalization of the previous method, with the difference that the state of neuron excitation is given by an internal potential in the form of the interval $[x, y]$, which corresponds again to its own excitation in the form of an interval defined by $[a, b]$. If we denote the set of neurons of the preceding (lower) layer as 'low', then for the minimal or maximal internal potential and for a positive value of the neuron's steepness, the following holds:

$$
x = w_0 + \sum_{w_i > 0, i \in \text{low}} w_i a_i + \sum_{w_i < 0, i \in \text{low}} w_i b_i,
$$

or respectively,

$$
y = w_0 + \sum_{w_i > 0, i \in \text{low}} w_i b_i + \sum_{w_i < 0, i \in \text{low}} w_i a_i,
$$

where $a_i, b_i$ represent the minimal and maximal state of excitation of the neurons in the previous layer.

The actual excitation is then given by applying an activation function, so:

$$
a = S(x), \quad b = S(y).
$$

The use of the backpropagation method requires differentiable functions that define the internal potential of the neuron. For this purpose, a "continuous signum function" of the following form is used:

$$
s(w) = \frac{1}{1 + e^{-w}}, \quad \widetilde{s}(w) = \frac{1}{1 + e^w},
$$

for which approximately holds:

- $s(w) = 1$ as $w \to +\infty $,
- $s(w) = 0$ as $w \to -\infty $,
- $\widetilde{s}(w) = 1$ as $w \to -\infty$,
- $\widetilde{s}(w) = 0$ as $w \to +\infty$.

Using these functions, the relationships for the internal potential of the neuron can be rewritten in this way:

$$
x = w_0 + \sum_{i \in \text{low}} s(w_i) w_i a_i + \sum_{i \in \text{low}} \widetilde{s}(w_i) w_i b_i,
$$

$$
y = w_0 + \sum_{i \in \text{low}} s(w_i) w_i b_i + \sum_{i \in \text{low}} \widetilde{s}(w_i) w_i a_i.
$$

In this way, differentiability of the both expression is ensured. It is evident that $s(w) + \widetilde{s}(w) = 1$ for all $w$. This means that the value of $w_i$ is divided between $a_i$ and $b_i$ according to the degree of its positivity or negativity.

The last thing that needs to be resolved is the effect of the steepness of the neuron. It should be noted that $\widetilde{s}(w) = s(-w)$, and thus we can consider the steepness of the neuron $\lambda$ as the parameter for the signum function. The new form of the signum function will then be as follows:

$$
s(w) = \frac{1}{1 + e^{-\lambda w}}, \quad \widetilde{s}(w) = \frac{1}{1 + e^{\lambda w}}.
$$







### Implemenation in Python

**Import modules**

In [1]:
import numpy as np
import pandas as pd
from backpropagation import IntervalNeuralNet


**Create training set from diagnosis.xlsx**

In [2]:
data = pd.read_excel('diagnosis.xlsx')
data.head()
training_set = []
for index, row in data.iterrows():
    features = [row['Fever'], row['Cough'], row['Headache'], row['Tiredness'], row['Night Sweat']]
    result = [row['Pneumonia'], row['Flu'], row['Cold']]
    training_set.append((features, result))
    
training_set

[([0.0, 0.5, 0.5, 0.0, 0.0], [0.0, 0.0, 1.0]),
 ([1.0, 1.0, 1.0, 1.0, 0.0], [0.0, 1.0, 0.0]),
 ([0.5, 1.0, 0.0, 1.0, 1.0], [1.0, 0.0, 0.0]),
 ([0.0, 0.5, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0]),
 ([1.0, 0.5, 1.0, 0.5, 0.0], [0.0, 1.0, 0.0]),
 ([1.0, 1.0, 0.0, 0.5, 0.5], [1.0, 0.0, 0.0])]

**Initialize and learn multilayered neural network**
* training set is given by a list [([i11, i12 ... i1N],[o11 ... o1M]), ... ([ik1, ..., ikN],[ok1, ..., okM])]
* topology [num_of_features, num_of_inner_neurons, num_of_inner_neurons, num_of_results]
* learning rates [for_weights, for_biases, for_slopes]
* number of leraning epochs

In case of learning rates for biases and slopes equal 0 only weights based learning is executed

In [3]:
bpnn = IntervalNeuralNet(training_set,[5, 5, 5, 3], [0.3, 0.3, 0.3], 1000)
bpnn.backpropagation()
for row in training_set:
    net_input = row[0]
    bpnn.feed_forward(net_input)
    net_output = bpnn.output_activation.reshape([1,3])
    print("Input: ", net_input, " Output: ", net_output.flatten().tolist())




Input:  [0.0, 0.5, 0.5, 0.0, 0.0]  Output:  [0.0, 0.0, 0.99]
Input:  [1.0, 1.0, 1.0, 1.0, 0.0]  Output:  [0.01, 0.99, 0.01]
Input:  [0.5, 1.0, 0.0, 1.0, 1.0]  Output:  [0.99, 0.01, 0.0]
Input:  [0.0, 0.5, 0.0, 0.0, 0.0]  Output:  [0.01, 0.0, 0.99]
Input:  [1.0, 0.5, 1.0, 0.5, 0.0]  Output:  [0.01, 0.99, 0.01]
Input:  [1.0, 1.0, 0.0, 0.5, 0.5]  Output:  [0.99, 0.01, 0.0]


### Usage of diagnosis app
**Specification of the patient with uncertain input data**
| Attribute | Value |
|-----------|-------|
|   Fever  |  38.0 - 38.5   |
|   Cough  |   1  |
|   Headache  |   1  |
|   Tiredness  |   1  |
|   Night Sweat  |  unknown |

In [4]:
input = [0.6, 1, 1, 1, 0]
output = bpnn.run(input)
output

[0.01, 0.99, 0.01]

In [5]:
input = [(0.5, 0.7), 1, 1, 1, (0, 0)]
output = bpnn.interval_run(input)
output


[(0.5, 0.7), (1, 1), (1, 1), (1, 1), (0, 0)]